# Install Library yang dibutuhkan

In [1]:
pip install tfx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.5/173.5 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
IN

In [2]:
pip show tfx

ERROR: Operation cancelled by user


In [ ]:
pip show tensorflow

In [ ]:
pip install tensorflow --upgrade

In [ ]:
pip install tensorflow-model-analysis==0.46.0

In [ ]:
pip install tensorflow==2.15

# Import Library

In [1]:
# Import library
import os
from typing import Text
from absl import logging
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner

Melakukan set variabel seperti pipeline name, path untuk menyimpan output, path module, dan banyak lainnya.

In [2]:
# Nama pipeline
PIPELINE_NAME = "heart-disease-pipeline"

# Pipeline inputs
DATA_ROOT = "/content/data"
TRANSFORM_MODULE_FILE = "/content/modules/transform.py"
TUNER_MODULE_FILE = "/content/modules/tuner.py"
TRAINER_MODULE_FILE = "/content/modules/trainer.py"
COMPONENTS_MODULE_FILE = "/content/modules/components.py"

# Pipeline outputs
OUTPUT_BASE = "output"
serving_model_dir = os.path.join(OUTPUT_BASE, 'serving_model')
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

Pembuatan pipeline component module file menggunakan magic command. Pipeline terdiri dari:

1. CsvExampleGen
2. StatisticsGen
3. SchemaGen
4. ExampleValidator
5. Transform
6. Trainer
7. Evaluator
8. Pusher

Komponen trainer sudah menggunakan komponen tuner. Pusher akan melakukan push model jika melebihi syarat dari BinaryAccuracy 0.5

# Membuat Pipeline Components

In [3]:
%%writefile {COMPONENTS_MODULE_FILE}

# Import library
import os
import tensorflow_model_analysis as tfma
from tfx.components import (
    CsvExampleGen,
    StatisticsGen,
    SchemaGen,
    ExampleValidator,
    Transform,
    Tuner,
    Trainer,
    Evaluator,
    Pusher
)
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.types import Channel
from tfx.dsl.components.common.resolver import Resolver
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import (
    LatestBlessedModelStrategy)

# Fungsi untuk melakukan inisialisasi components
def init_components(config):

    """Returns tfx components for the pipeline.

    Args:
        data_dir (str): Directory containing the dataset.
        transform_module (str): Path to the transform module.
        tuner_module (str): Path to the tuner module.
        training_module (str): Path to the training module.
        training_steps (int): Number of training steps.
        eval_steps (int): Number of evaluation steps.
        serving_model_dir (str): Directory to save the serving

    Returns:
        components: Tuple of TFX components.
    """

    # Membagi dataset dengan perbandingan 8:2
    output = example_gen_pb2.Output(
        split_config = example_gen_pb2.SplitConfig(splits=[
            example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
            example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
        ])
    )

    # Komponen example gen
    example_gen = CsvExampleGen(
        input_base=config["DATA_ROOT"],
        output_config=output
    )

    # Komponen statistics gen
    statistics_gen = StatisticsGen(
        examples=example_gen.outputs["examples"]
    )

    # Komponen schema gen
    schema_gen = SchemaGen(
        statistics=statistics_gen.outputs["statistics"]
    )

    # Komponen example validator
    example_validator = ExampleValidator(
        statistics=statistics_gen.outputs['statistics'],
        schema=schema_gen.outputs['schema']
    )

    # Komponen transform. Menggunakan module transform.py
    transform  = Transform(
        examples=example_gen.outputs['examples'],
        schema= schema_gen.outputs['schema'],
        module_file=os.path.abspath(config["transform_module"])
    )

    # Komponen tuner. Menggunakan module tuner.py
    tuner = Tuner(
        module_file=os.path.abspath(config["tuner_module"]),
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        schema=schema_gen.outputs['schema'],
        train_args=trainer_pb2.TrainArgs(
            splits=['train'],
            num_steps=config["training_steps"]),
        eval_args=trainer_pb2.EvalArgs(
            splits=['eval'],
            num_steps=config["eval_steps"]),
    )

    # Komponen trainer. Menggunakan module trainer.py
    trainer  = Trainer(
        module_file=os.path.abspath(config["training_module"]),
        examples = transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        schema=schema_gen.outputs['schema'],
        hyperparameters=tuner.outputs['best_hyperparameters'],
        train_args=trainer_pb2.TrainArgs(
            splits=['train'],
            num_steps=config["training_steps"]),
        eval_args=trainer_pb2.EvalArgs(
            splits=['eval'],
            num_steps=config["eval_steps"])
    )

    # Komponen model resolver
    model_resolver = Resolver(
        strategy_class= LatestBlessedModelStrategy,
        model = Channel(type=Model),
        model_blessing = Channel(type=ModelBlessing)
    ).with_id('Latest_blessed_model_resolver')

    metrics_specs = [
        tfma.MetricsSpec(metrics=[
                tfma.MetricConfig(class_name='AUC'),
                tfma.MetricConfig(class_name="Precision"),
                tfma.MetricConfig(class_name="Recall"),
                tfma.MetricConfig(class_name="ExampleCount"),
                tfma.MetricConfig(class_name='BinaryAccuracy',
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={'value':0.8}),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={'value':0.0001})
                        )
                )
            ])
    ]


    eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='target')],  # Ensure 'target' is the correct label
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=metrics_specs
    )


    # Komponen evaluator
    evaluator = Evaluator(
        examples=example_gen.outputs['examples'],
        model=trainer.outputs['model'],
        baseline_model=model_resolver.outputs['model'],
        eval_config=eval_config)

    # Komponen pusher
    pusher = Pusher(
        model=trainer.outputs["model"],
        model_blessing=evaluator.outputs["blessing"],
        push_destination=pusher_pb2.PushDestination(
            filesystem=pusher_pb2.PushDestination.Filesystem(
                base_directory=config["serving_model_dir"]
            )
        ),
    )

    # Mengembalikan semua komponen
    components = (
        example_gen,
        statistics_gen,
        schema_gen,
        example_validator,
        transform,
        tuner,
        trainer,
        model_resolver,
        evaluator,
        pusher
    )

    # Mengembalikan komponen
    return components

Overwriting /content/modules/components.py


# Data Transform

In [4]:
%%writefile {TRANSFORM_MODULE_FILE}

# Import library
import tensorflow as tf
import tensorflow_transform as tft

LABEL_KEY = "target"

# List of feature names and types
NUMERIC_FEATURES = ['age', 'ca', 'chol', 'oldpeak', 'thalach', 'trestbps']
CATEGORICAL_FEATURES = ['cp', 'exang', 'fbs', 'restecg', 'sex', 'slope', 'thal']

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features.

    Args:
        inputs: dictionary of raw input features.

    Returns:
        outputs: dictionary of transformed features.
    """
    outputs = {}

    # Filter out rows with invalid values for 'ca' and 'thal'
    valid_rows = tf.logical_and(
        tf.not_equal(inputs['ca'], 4),  # Exclude rows where `ca` is 4
        tf.not_equal(inputs['thal'], 0)  # Exclude rows where `thal` is 0
    )

    # Apply the valid row mask to all inputs
    filtered_inputs = {key: tf.boolean_mask(inputs[key], valid_rows) for key in inputs}

    # Normalize numeric features
    for feature in NUMERIC_FEATURES:
        outputs[transformed_name(feature)] = tft.scale_to_z_score(filtered_inputs[feature])

    # Label encode categorical features
    for feature in CATEGORICAL_FEATURES:
        outputs[transformed_name(feature)] = tft.compute_and_apply_vocabulary(filtered_inputs[feature])

    # Include the label
    outputs[LABEL_KEY] = filtered_inputs[LABEL_KEY]

    return outputs

Overwriting /content/modules/transform.py


Kode di atas adalah sebuah modul pemrosesan data untuk pra-pemrosesan fitur pada dataset menggunakan TensorFlow Transform (TFT). Berikut adalah penjelasan singkat dari proses yang dilakukan:

1. Fitur Numerik: Fitur numerik seperti usia, kolesterol, dan tekanan darah diubah agar memiliki distribusi yang lebih baik dengan melakukan normalisasi menggunakan teknik z-score (standarisasi).

2. Fitur Kategorikal: Fitur kategorikal seperti jenis kelamin dan hasil tes kesehatan lainnya diberi representasi angka (label encoding) berdasarkan frekuensi kemunculan kategori tersebut dalam dataset.

3. Filter Data Tidak Valid: Beberapa baris data yang memiliki nilai tidak valid, seperti nilai ca yang bernilai 4 atau thal yang bernilai 0, dihapus dari dataset.

4. Fungsi Utama: Fungsi preprocessing_fn mengatur semua langkah pemrosesan data ini dengan menerima data mentah sebagai input dan mengembalikan data yang telah diproses sesuai dengan transformasi yang telah ditentukan.

Proses ini penting untuk membersihkan dan menyiapkan data sebelum digunakan dalam model machine learning.

In [5]:
%%writefile {TUNER_MODULE_FILE}

# Import library
import tensorflow as tf
import os
import kerastuner as kt
import tensorflow_transform as tft
import keras_tuner as kt
from tfx.v1.components import TunerFnResult
from tfx.components.trainer.fn_args_utils import FnArgs

# Definisikan nama label
LABEL_KEY = "target"

# Daftar fitur numerik dan kategorikal
NUMERIC_FEATURES = ['age', 'ca', 'chol', 'oldpeak', 'thalach', 'trestbps']
CATEGORICAL_FEATURES = ['cp', 'exang', 'fbs', 'restecg', 'sex', 'slope', 'thal']

def transformed_name(key):
    """Menambahkan suffix '_xf' pada nama fitur yang telah ditransformasi"""
    return key + "_xf"

# Fungsi untuk membaca data yang telah di-compress
def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

# Fungsi untuk membuat model
def model_builder(hyperparameters):
    """
    This function defines a Keras model and returns the model as a
    Keras object.
    """

    input_features = []

    # Menambahkan input layer untuk setiap fitur
    for feature in NUMERIC_FEATURES + CATEGORICAL_FEATURES:
        input_features.append(tf.keras.Input(shape=(1,), name=transformed_name(feature)))

    concatenate = tf.keras.layers.concatenate(input_features)

    deep = tf.keras.layers.Dense(hyperparameters.Choice(
        'unit_1', [8,16]),
        activation="relu")(concatenate)
    deep = tf.keras.layers.Dense(hyperparameters.Choice(
        'unit_2', [16,32]),
        activation="relu")(deep)
    deep = tf.keras.layers.Dense(hyperparameters.Choice(
        'unit_3', [16,32]),
        activation="relu")(deep)
    deep = tf.keras.layers.Dense(hyperparameters.Choice(
        'unit_4', [32,64]),
        activation="relu")(deep)

    outputs = tf.keras.layers.Dense(1, activation="sigmoid")(deep)

    model = tf.keras.models.Model(inputs=input_features, outputs=outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hyperparameters.Choice(
                'learning_rate', [0.01,0.001])),
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

    return model

# Fungsi input untuk mempersiapkan dataset
def input_fn(file_pattern, tf_transform_output, batch_size=64):
    # Mendapatkan feature_spec untuk fitur yang sudah ditransformasi
    transform_feature_spec = tf_transform_output.transformed_feature_spec().copy()

    # Membaca data dalam bentuk batch
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        label_key='target'  # Menggunakan 'target' sebagai label key yang benar
    )

    # Fungsi untuk format data (menyesuaikan label dan fitur)
    def format_data(features, labels):
        labels = tf.reshape(labels, [-1, 1])  # Bentuk label sesuai dengan output
        return features, labels

    return dataset.map(format_data)

# Fungsi tuner_fn
def tuner_fn(fn_args: FnArgs):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    train_dataset = input_fn(fn_args.train_files, tf_transform_output, batch_size=10)
    eval_dataset = input_fn(fn_args.eval_files, tf_transform_output, batch_size=10)

    tuner = kt.RandomSearch(
        model_builder,
        objective=kt.Objective("val_binary_accuracy", direction="max"),
        max_trials=10,
        directory=os.path.join(fn_args.working_dir, 'tuner'),
        project_name='kt_random_search'
    )

    tuner.search_space_summary()

    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            "x": train_dataset,
            'validation_data': eval_dataset,
            'steps_per_epoch': fn_args.train_steps,
            'validation_steps': fn_args.eval_steps,
            "epochs": 10
        }
    )

Overwriting /content/modules/tuner.py


# Model Development

In [6]:
%%writefile {TRAINER_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft
from tfx.components.trainer.fn_args_utils import FnArgs
from keras.utils import plot_model

LABEL_KEY = "target"
NUMERIC_FEATURES = ['age', 'ca', 'chol', 'oldpeak', 'thalach', 'trestbps']
CATEGORICAL_FEATURES = ['cp', 'exang', 'fbs', 'restecg', 'sex', 'slope', 'thal']

def transformed_name(key):
    return key + "_xf"

def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, batch_size=64):
    transform_feature_spec = tf_transform_output.transformed_feature_spec().copy()

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        label_key=LABEL_KEY
    )

    def format_data(features, labels):
        labels = tf.reshape(labels, [-1, 1])
        return features, labels

    return dataset.map(format_data)

def build_keras_model(hparams):
    input_features = []
    for feature in NUMERIC_FEATURES + CATEGORICAL_FEATURES:
        input_features.append(tf.keras.Input(shape=(1,), name=transformed_name(feature)))

    concatenate = tf.keras.layers.concatenate(input_features)

    deep = tf.keras.layers.Dense(hparams['unit_1'], activation="relu")(concatenate)
    deep = tf.keras.layers.Dense(hparams['unit_2'], activation="relu")(deep)
    deep = tf.keras.layers.Dense(hparams['unit_3'], activation="relu")(deep)
    deep = tf.keras.layers.Dense(hparams['unit_4'], activation="relu")(deep)

    outputs = tf.keras.layers.Dense(1, activation="sigmoid")(deep)

    model = tf.keras.Model(inputs=input_features, outputs=outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hparams['learning_rate']),
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

    return model

# Fungsi untuk menyajikan TF examples
def _get_serve_tf_examples_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)

        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)

        return model(transformed_features)

    return serve_tf_examples_fn

# Fungsi untuk mendapatkan signature dari fitur transformasi
def _get_transform_features_signature(model, tf_transform_output):
    model.tft_layer_eval = tf_transform_output.transform_features_layer()

    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
    ])
    def transform_features_fn(serialized_tf_example):
        raw_feature_spec = tf_transform_output.raw_feature_spec()
        raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
        transformed_features = model.tft_layer_eval(raw_features)
        return transformed_features

    return transform_features_fn

def run_fn(fn_args: FnArgs):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    train_dataset = input_fn(fn_args.train_files, tf_transform_output)
    eval_dataset = input_fn(fn_args.eval_files, tf_transform_output)

    best_hyperparameters = fn_args.hyperparameters.get('values')

    model = build_keras_model(best_hyperparameters)

    model.fit(
        train_dataset,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps,
        epochs=10
    )

    signatures = {
        'serving_default': _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
            tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')),
        'transform_features': _get_transform_features_signature(model, tf_transform_output),
    }

    tf.saved_model.save(model, fn_args.serving_model_dir, signatures=signatures)

    plot_model(
        model,
        to_file='images/model_plot.png',
        show_shapes=True,
        show_layer_names=True
    )

Overwriting /content/modules/trainer.py


# Melakukan Inisialisasi Local Pipeline.

In [7]:
def init_local_pipeline(
    components, pipeline_root: Text
) -> pipeline.Pipeline:

    logging.info(f"Pipeline root set to: {pipeline_root}")
    beam_args = [
        "--direct_running_mode=multi_processing",
        # 0 auto-detect based on on the number of CPUs available
        # during execution time.
        "----direct_num_workers=0"
    ]

    return pipeline.Pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=pipeline_root,
        components=components,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            metadata_path
        ),
        beam_pipeline_args=beam_args
    )

# Menjalankan Pipeline Menggunakan Apache Beam.

In [8]:
from modules.components import init_components

logging.set_verbosity(logging.INFO)

config = {
    "DATA_ROOT": DATA_ROOT,
    "tuner_module": TUNER_MODULE_FILE,
    "training_module": TRAINER_MODULE_FILE,
    "transform_module": TRANSFORM_MODULE_FILE,
    "training_steps": 1000,
    "eval_steps": 250,
    "serving_model_dir": serving_model_dir,
}

components = init_components(config)

pipeline = init_local_pipeline(components, pipeline_root)
BeamDagRunner().run(pipeline=pipeline)

INFO:absl:Finished tuning... Tuner ID: tuner0
INFO:absl:Best HyperParameters: {'space': [{'class_name': 'Choice', 'config': {'name': 'unit_1', 'default': 8, 'conditions': [], 'values': [8, 16], 'ordered': True}}, {'class_name': 'Choice', 'config': {'name': 'unit_2', 'default': 16, 'conditions': [], 'values': [16, 32], 'ordered': True}}, {'class_name': 'Choice', 'config': {'name': 'unit_3', 'default': 16, 'conditions': [], 'values': [16, 32], 'ordered': True}}, {'class_name': 'Choice', 'config': {'name': 'unit_4', 'default': 32, 'conditions': [], 'values': [32, 64], 'ordered': True}}, {'class_name': 'Choice', 'config': {'name': 'learning_rate', 'default': 0.01, 'conditions': [], 'values': [0.01, 0.001], 'ordered': True}}], 'values': {'unit_1': 8, 'unit_2': 16, 'unit_3': 16, 'unit_4': 32, 'learning_rate': 0.01}}
INFO:absl:Best Hyperparameters are written to output/heart-disease-pipeline/Tuner/best_hyperparameters/117/best_hyperparameters.txt.
INFO:absl:Tuner results are written to output

Trial 10 Complete [00h 00m 47s]
val_binary_accuracy: 0.8435999751091003

Best val_binary_accuracy So Far: 0.8632000088691711
Total elapsed time: 00h 07m 05s
Results summary
Results in output/heart-disease-pipeline/Tuner/.system/executor_execution/117/.temp/117/tuner/kt_random_search
Showing 10 best trials
Objective(name="val_binary_accuracy", direction="max")

Trial 01 summary
Hyperparameters:
unit_1: 8
unit_2: 16
unit_3: 16
unit_4: 32
learning_rate: 0.01
Score: 0.8632000088691711

Trial 07 summary
Hyperparameters:
unit_1: 8
unit_2: 32
unit_3: 16
unit_4: 32
learning_rate: 0.001
Score: 0.8628000020980835

Trial 08 summary
Hyperparameters:
unit_1: 16
unit_2: 32
unit_3: 16
unit_4: 32
learning_rate: 0.001
Score: 0.8435999751091003

Trial 09 summary
Hyperparameters:
unit_1: 16
unit_2: 32
unit_3: 16
unit_4: 64
learning_rate: 0.001
Score: 0.8435999751091003

Trial 04 summary
Hyperparameters:
unit_1: 16
unit_2: 16
unit_3: 32
unit_4: 32
learning_rate: 0.001
Score: 0.8432000279426575

Trial 02 s

INFO:absl:node Trainer is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.components.trainer.component.Trainer"
    base_type: TRAIN
  }
  id: "Trainer"
}
contexts {
  contexts {
    type {
      name: "pipeline"
    }
    name {
      field_value {
        string_value: "heart-disease-pipeline"
      }
    }
  }
  contexts {
    type {
      name: "pipeline_run"
    }
    name {
      field_value {
        string_value: "20241225-163901.665857"
      }
    }
  }
  contexts {
    type {
      name: "node"
    }
    name {
      field_value {
        string_value: "heart-disease-pipeline.Trainer"
      }
    }
  }
}
inputs {
  inputs {
    key: "examples"
    value {
      channels {
        producer_node_query {
          id: "Transform"
        }
        context_queries {
          type {
            name: "pipeline"
          }
          name {
            field_value {
              string_value: "heart-disease-pipeline"
            }
          }
        

Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - binary_accuracy: 0.9315 - loss: 0.1610 - val_binary_accuracy: 0.8235 - val_loss: 3.8471
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - binary_accuracy: 0.9830 - loss: 0.0623 - val_binary_accuracy: 0.8627 - val_loss: 4.4017
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - binary_accuracy: 0.9998 - loss: 0.0011 - val_binary_accuracy: 0.8432 - val_loss: 5.6661
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - binary_accuracy: 1.0000 - loss: 7.8000e-05 - val_binary_accuracy: 0.8431 - val_loss: 6.2677
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - binary_accuracy: 1.0000 - loss: 2.4606e-05 - val_binary_accuracy: 0.8431 - val_loss: 6.6948
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - binary_accuracy: 1.0000 - loss: 1.0512e-05 - val_binary_accuracy: 0.8432 - val_loss: 7.0325
Epoch 7/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - binary_accuracy: 1.0000 - loss: 4.9662e-06 - val_binary_accuracy: 0.

INFO:absl:Feature age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature ca has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature chol has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature cp has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature exang has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature fbs has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature oldpeak has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature restecg has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature sex has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature slope has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature target has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature thal has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature thalach has a shape dim {
  siz

In [9]:
# Tulis isi Dockerfile
dockerfile_content = """
FROM tensorflow/serving:latest

COPY ./output/serving_model /models/cc-model
COPY ./config /model_config
ENV MODEL_NAME=cc-model

ENV MONITORING_CONFIG="/model_config/prometheus.config"
ENV PORT=8501
RUN echo '#!/bin/bash \n\n\
env \n\
tensorflow_model_server --port=8500 --rest_api_port=${PORT} \
--model_name=${MODEL_NAME} --model_base_path=${MODEL_BASE_PATH}/${MODEL_NAME} \
--monitoring_config_file=${MONITORING_CONFIG} \
"$@"' > /usr/bin/tf_serving_entrypoint.sh \
&& chmod +x /usr/bin/tf_serving_entrypoint.sh
"""

# Simpan sebagai file Dockerfile
with open("Dockerfile", "w") as f:
    f.write(dockerfile_content)

print("Dockerfile has been written.")

Dockerfile has been written.


In [11]:
# Tulis isi Dockerfile
dockerfile_content = """
FROM prom/prometheus:latest

COPY prometheus.yml /etc/prometheus/prometheus.yml
"""

# Simpan sebagai file Dockerfile
with open("/content/monitoring/Dockerfile", "w") as f:
    f.write(dockerfile_content)

print("Dockerfile has been written.")

Dockerfile has been written.


In [12]:
# Tulis isi yml
yml_content = """
global:
  scrape_interval: 5s
  evaluation_interval: 5s
  external_labels:
    monitor: "tf-serving-monitor"

scrape_configs:
  - job_name: "prometheus"
    scrape_interval: 5s
    metrics_path: /monitoring/prometheus/metrics
    scheme: "https"
    static_configs:
      - targets: ['proyek-akhir-mlops-production.up.railway.app']
"""

# Simpan sebagai file Dockerfile
with open("/content/monitoring/prometheus.yml", "w") as f:
    f.write(yml_content)

print("prometheus has been written.")

prometheus has been written.


In [13]:
# Tulis isi config
config_content = """
prometheus_config {
   enable: true,
   path: "/monitoring/prometheus/metrics"
}
"""

# Simpan sebagai file Dockerfile
with open("/content/config/prometheus.config", "w") as f:
    f.write(config_content)

print("prometheus has been written.")

prometheus has been written.


In [14]:
import shutil

# Path to the directory you want to zip
dir_path = '/content'

# Path where the zip file will be saved
output_zip_path = '/content/data.zip'

# Create a zip file
shutil.make_archive(output_zip_path.replace('.zip', ''), 'zip', dir_path)

'/content/data.zip'

# Test

In [15]:
import tensorflow as tf
import json
import requests
import base64

# Fungsi untuk membuat serialized example
def create_serialized_example(inputs):
    feature_mapping = {
        "age": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs["age"])])),
        "sex": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs["sex"])])),
        "cp": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs["cp"])])),
        "trestbps": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs["trestbps"])])),
        "chol": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs["chol"])])),
        "fbs": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs["fbs"])])),
        "restecg": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs["restecg"])])),
        "thalach": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs["thalach"])])),
        "exang": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs["exang"])])),
        "oldpeak": tf.train.Feature(float_list=tf.train.FloatList(value=[float(inputs["oldpeak"])])),
        "slope": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs["slope"])])),
        "ca": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs["ca"])])),
        "thal": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs["thal"])])),
    }

    # Membuat tf.train.Example
    example = tf.train.Example(features=tf.train.Features(feature=feature_mapping))
    return example.SerializeToString()

# URL endpoint TensorFlow Serving
url = "https://tugas-akhir-mlops-production.up.railway.app/v1/models/cc-model:predict"

# Data input
inputs = {
    "age": 63,
    "sex": 1,
    "cp": 3,
    "trestbps": 145,
    "chol": 233,
    "fbs": 1,
    "restecg": 0,
    "thalach": 150,
    "exang": 0,
    "oldpeak": 2.3,
    "slope": 0,
    "ca": 0,
    "thal": 1,
}

# Membuat serialized example
serialized_example = create_serialized_example(inputs)

# Mengonversi serialized example ke base64
serialized_example_base64 = base64.b64encode(serialized_example).decode('utf-8')

# Membuat payload untuk request
data = {
    "signature_name": "serving_default",  # Sesuaikan dengan signature pada model
    "instances": [{"b64": serialized_example_base64}],  # Mengirim serialized example dalam format base64
}

# Mengirimkan request POST ke server
response = requests.post(url, json=data)

# Menangani response
if response.status_code == 200:
    predictions = response.json()
    predicted_label = "Positive" if predictions["predictions"][0][0] > 0.5 else "Negative"
    print(f"Predicted Label: {predicted_label}")
else:
    print(f"Error: {response.status_code} - {response.text}")


Predicted Label: Positive
